# RTMA example

The ETo/ETr calculation for recent RTMA images have been updated to follow the equations in ASCE 2005.

Test that the precomputed ETr in the collection matches wih the ETr generated using the eerefet module. 

In [1]:
import pprint
import ee
import openet.refetgee

ee.Initialize()

#### Load RTMA for a single date

In [2]:
# Build date range to match GRIDMET "day"
start_date = ee.Date.fromYMD(2018, 4, 1).advance(6, 'hour')
end_date = start_date.advance(1, 'day')
input_coll = ee.ImageCollection('NOAA/NWS/RTMA')\
    .filterDate(start_date, end_date)

# input_coll = ee.ImageCollection('NOAA/NWS/RTMA')\
#     .filterDate('2018-04-01', '2018-04-02')
    
# input_img = ee.Image(ee.ImageCollection(input_coll.first()))
input_img = ee.Image('NOAA/NWS/RTMA/2018040118')

#### Ancillary data and test point

In [3]:
test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match source elevation grid exactly
elev_img = ee.Image("projects/climate-engine/rtma/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the source images
crs = input_img.projection().getInfo()['wkt']
geo = input_img.projection().getInfo()['transform']

In [4]:
# GRIDMET solar radiation (in MJ m-2 day-1)
rs_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')\
    .filterDate(start_date, end_date)\
    .select(['srad'])
rs_img = ee.Image(rs_coll.first()).multiply(0.0864)
rs = rs_img.reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo).getInfo()['srad']
print(rs)
# print(rs / 0.0864)

23.4144


#### Compute ETr for a single hour

This uses the NLDAS hourly incoming solar radiation

In [5]:
print(openet.refetgee.Hourly.rtma(input_img, elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

0.571071156941792


#### Compute daily ETr from the aggregated hourly inputs

This uses the GRIDMET daily solar

In [6]:
print(openet.refetgee.Daily.rtma(input_coll, elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

6.93926120145355


#### Compute daily ETr as the sum of the hourly ETr

This uses the NLDAS hourly solar

In [7]:
def hourly_etr(img):
    return ee.Image(openet.refetgee.Hourly.rtma(img, elev=elev_img, lat=lat_img).etr)\
        .copyProperties(img, ['system:time_start'])
etr = ee.Image(ee.ImageCollection(input_coll.map(hourly_etr)).sum())\
    .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etr.getInfo()['etr'])

6.391883119555344


#### Check that the different "rs" parameters all work

In [8]:
print(openet.refetgee.Daily.rtma(input_coll, rs='GRIDMET', elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

6.93926120145355


In [9]:
print(openet.refetgee.Daily.rtma(input_coll, rs='NLDAS', elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

6.937925792265972


In [10]:
print(openet.refetgee.Daily.rtma(input_coll, rs=rs_img, elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

6.93926120145355


In [11]:
print(openet.refetgee.Daily.rtma(input_coll, rs=rs, elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

6.93926120145355


In [12]:
print(openet.refetgee.Daily.rtma(input_coll, rs=ee.Number(rs), elev=elev_img, lat=lat_img).etr\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etr'])

6.93926120145355
